In [1]:
import platform
import re
version=platform.python_version()
ISPY2=re.match('^2.',version)
print('ISPY2',ISPY2 is not None)
import os,sys
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
sys.path.append(os.environ['PROJECT_DIR'])
if not ISPY2:
    from importlib import reload

ISPY2 False


In [50]:
import re
import math
import numpy as np
import geopandas as gpd
import pandas as pd
import geojson 
from config import REGIONS_DIR
from glob import glob
from pprint import pprint
import descarteslabs as dl
import utils.load as load
import utils.helpers as h
import ulu.info as info
import dl_jobs.nd_json as ndj
import mproc

---

In [36]:
def tiles_path(i):
    return f'{PATH_ROOT}-b{i}.p'


def get_tile_keys(name,i,shape):
    try:
        tiles=dl.scenes.DLTile.from_shape(
                shape=shape, 
                resolution=res, 
                tilesize=size, 
                pad=pad )
        return [t.key for t in tiles]
    except Exception as e:
        print("ERROR:",i,name,e)
        return []



def save_features(name_feats,limit=None):
    name,features=name_feats
    print("\nSTART:",name)
    if limit:
        features=features[:limit]
    def _run():
        tiles=[t for t in (get_tile_keys(name,i,f) for i,f in enumerate(features))]
        tiles=h.flatten_list(tiles)
        h.save_pickle(tiles,name)
        return len(tiles)
    count=_run()
    print("\nCOMPLETE:",count,name)
    return name

---

In [4]:
PRODUCT='test_prod'
REGION='urban_india'
LIMIT=None

In [5]:
path=info.get_tiles_path(PRODUCT,REGION,LIMIT)
meta=load.meta(PRODUCT)
res,size,pad=h.resolution_size_padding(meta=meta)

In [7]:
PATH_ROOT=re.sub('.p$','',path)
SORTED_PATH=f"{PATH_ROOT.split('/')[-1]}.sorted-unique.geojson"
SORTED_PATH

'urban_india-5:1024:8.sorted-unique.geojson'

---

In [8]:
# %time gdf=load.geodataframe(f'{REGION}.geojson')

In [9]:
# gdf['lon']=gdf.geometry.centroid.x
# gdf['lat']=gdf.geometry.centroid.y
# gdf.sort_values(by=['lon','lat'],axis=0,inplace=True)
# gdf.reset_index(inplace=True,drop=True)
# print("NB_SAMPLES:",gdf.shape[0])
# gdf.head(3)

In [10]:
# gdf.Urban.unique()
# gdf.drop(['Urban'],axis=1,inplace=True)
# gdf.sample()

In [11]:
# gdf.to_file(SORTED_PATH, driver='GeoJSON')

---

In [12]:
gdf=gpd.read_file(SORTED_PATH)
gdf.shape[0]

26869

In [13]:
%time feature_collection=geojson.loads(gdf.to_json())
features=feature_collection['features']
del(gdf)
del(feature_collection)

CPU times: user 38.7 s, sys: 1.33 s, total: 40 s
Wall time: 40 s


In [16]:
TOTAL=len(features)
N=5
BS=int(math.ceil(TOTAL/N))
BATCHES=[(tiles_path(n),features[n*BS:(n+1)*BS]) for n in range(N)]
print("BATCH_SIZE,NB_BATHES,TOTAL: ",BS,N,TOTAL)
for path,feats in BATCHES:
    print('-',len(feats),path)

BATCH_SIZE,NB_BATHES,TOTAL:  5374 5 26869
- 5374 /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b0.p
- 5374 /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b1.p
- 5374 /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b2.p
- 5374 /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b3.p
- 5373 /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b4.p


---

In [38]:
# %time path=save_features(BATCHES[0],limit=15)

In [31]:
# tkeys=h.read_pickle(path)
# print(len(tkeys))
# tkeys[:5]

In [39]:
%time out=mproc.map_with_threadpool(save_features,BATCHES,max_processes=len(BATCHES))


START:
START: /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b1.p

START: /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b2.p

START: /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b3.p

START: /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b0.p
 /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b4.p

COMPLETE: 20369 /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b3.p

COMPLETE: 20465 /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b4.p
ERROR: 5098 /Users/brook/WRI/RW/code/ulu/ulu_pixelwise_tasks/repo/data/tiles/urban_india-5:1024:8-b1.p HTTPSConnectionPool(host='platform.descarteslabs.com', port=443): Max retries exceeded with url: /raster/v1/dlkeys/from_shape (Caused by ResponseError('too many 500 error responses',))



In [41]:
for index in range(5):
    print(index,len(h.read_pickle(tiles_path(index))))

0 19609
1 19771
2 19178
3 20369
4 20465


In [48]:
# p,feats=BATCHES[1]
# print(p)
# print(len(feats))
# bad_tile=feats[5098]
# %time tile_keys=get_tile_keys('feat_b1_5098',5098,bad_tile)
# h.save_pickle(tile_keys,f'{PATH_ROOT}-feat_b1_5098.p')

---

In [51]:
%time all_tiles=[h.read_pickle(tiles_path(i)) for i in range(N)]
all_tiles=h.flatten_list(all_tiles)
print('all',len(all_tiles))
df=pd.DataFrame(all_tiles,columns=['tile_key'])
print('\t=',df.shape[0])
unique_tiles=list(df.tile_key.unique())
print('unique',len(unique_tiles))
h.save_pickle(unique_tiles,f'{PATH_ROOT}-all.p')
tile_keys=h.read_pickle(f'{PATH_ROOT}-all.p')
print('\t=',len(tile_keys))

CPU times: user 12.3 ms, sys: 3.75 ms, total: 16.1 ms
Wall time: 14.9 ms
all 99392
	= 99392
unique 66202
	= 66202


In [52]:
DEV_COUNT=2000
tk=tile_keys[:DEV_COUNT]
path=f'{PATH_ROOT}-DEV{DEV_COUNT}.p'
print(f'{"dev"}[{len(tk)}]: {os.path.basename(path)}')
h.save_pickle(tk,path)

dev[2000]: urban_india-5:1024:8-DEV2000.p


In [54]:
DEV_COUNT=660
tk=tile_keys[:DEV_COUNT]
path=f'{PATH_ROOT}-DEV{DEV_COUNT}.p'
print(f'{"dev"}[{len(tk)}]: {os.path.basename(path)}')
h.save_pickle(tk,path)

dev[660]: urban_india-5:1024:8-DEV660.p
